In [5]:
# Import modules
import matplotlib.pyplot as plt
import numpy as np
from numpy import nan
import xarray as xr
import pandas as pd
import glob
import os
#import pygmt
import math
import scipy as sp
import sympy as smp 
from numpy.linalg import norm

In [6]:
horizontal = np.linspace(-50,50,101)     # x-coordinates
vertical = np.arange(50.,-0.1,-1)       # y-coordinates

'''calculating initial pressure'''
#P = np.concatenate((np.repeat(0,30),np.arange(0.,10.25,0.25),np.repeat(0,30)))
P = 8

horizontal, vertical


(array([-50., -49., -48., -47., -46., -45., -44., -43., -42., -41., -40.,
        -39., -38., -37., -36., -35., -34., -33., -32., -31., -30., -29.,
        -28., -27., -26., -25., -24., -23., -22., -21., -20., -19., -18.,
        -17., -16., -15., -14., -13., -12., -11., -10.,  -9.,  -8.,  -7.,
         -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.,   1.,   2.,   3.,   4.,
          5.,   6.,   7.,   8.,   9.,  10.,  11.,  12.,  13.,  14.,  15.,
         16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,  24.,  25.,  26.,
         27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,
         38.,  39.,  40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,
         49.,  50.]),
 array([50., 49., 48., 47., 46., 45., 44., 43., 42., 41., 40., 39., 38.,
        37., 36., 35., 34., 33., 32., 31., 30., 29., 28., 27., 26., 25.,
        24., 23., 22., 21., 20., 19., 18., 17., 16., 15., 14., 13., 12.,
        11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.]))

In [9]:
'''### without function - no integration - as same as DAHM 2000 ###'''

x = smp.symbols('x', real=True)
#f1 = smp.sin(x)* smp.sin(x)
#f2 = smp.cos(x)* smp.cos(x)
#f3 = smp.sin(2*x)
'''making symmetric model'''

result_directory_1 = "//home/prajakta_mohite/Data/DAHM2000/tests/pyramid_100X50/"

all_files_1 = glob.glob(os.path.join(result_directory_1,'INPUT*.dat'))
for f in all_files_1:
    os.remove(f)
    
    
for i in range(len(vertical)): 
    for j in range(len(horizontal)): 
        A = horizontal[j] 
        B = vertical[i]
        x1,y1 = A,B        # P1 Any point on grid   - starting point of A
        x2,y2 = 0,B        # Normal                 - terminal of A
        x3,y3 = x1,y1      # P1 Any point on grid   - starting point of B
        x4,y4 = -20,0       # C1 (-4.0)              - could be terminal of B - B1
        x5,y5 = 0,0       # C2 (0,0)               - could be terminal of B - B2
        x6,y6 = 20,0       # C3 (4,0)               - could be terminal of B - B3
        
        sum1 = ((x2-x1) * (x4-x1)) + ((y2-y1) * (y4-y1))   # normal + negative
        sum2 = ((x2-x3) * (x5-x3)) + ((y2-y3) * (y5-y3))   # normal + zero
        sum3 = ((x2-x3) * (x6-x3)) + ((y2-y3) * (y6-y3))   # normal + positive 
        
        sum4 = np.sqrt(np.square(x2-x1) + np.square(y2-y1)) # P1 + normal
        
        sum5 = np.sqrt(np.square(x4-x3) + np.square(y4-y3))  # P1 + negative
        sum6 = np.sqrt(np.square(x5-x3) + np.square(y5-y3))  # P1 + zero
        sum7 = np.sqrt(np.square(x6-x3) + np.square(y6-y3))  # P1 + positive
        
        dot1 = np.dot(sum4,sum5)
        dot2 = np.dot(sum4,sum6)
        dot3 = np.dot(sum4,sum7)
        
        theta1 = np.arccos(sum1 / dot1)
        theta0 = np.arccos(sum2 / dot2)   
        theta2 = np.arccos(sum3 / dot3)
     
        r1 = math.dist((x1,y1),(x4,y4))      ### distance between P1 and C1
        r2 = math.dist((x1,y1),(x6,y6))      ### distance between P1 and C2
        ''' Integrating equation A1 from Gaetano'''
        if (r1 != 0 and r2 != 0):
            sxx = abs(P/2*np.pi*((1+x1/20)*(theta1-theta2)+np.sin(2*theta1)-((2*y1/20)*math.log(r2/r1))))
            syy = abs(P/2*np.pi*((1+x1/20)*(theta1-theta2)-np.sin(2*theta1)))
            sxy = (P/2*np.pi*((1-y1/20)*(theta1-theta2) - np.cos(2*theta1)))
        else:
            sxx = 0
            syy = 0
            sxy = 0

        '''Dahm 2000 Method'''
        '''with integration function'''
        #xx_stress = abs(2*p0/np.pi* smp.integrate(f1,(x,theta1,theta2)).simplify().evalf())
        #yy_stress = abs(2*p0/np.pi* smp.integrate(f2,(x,theta1,theta2)).simplify().evalf())
        #xy_stress = 2*p0/np.pi* smp.integrate(f3,(x,theta1,theta2)).simplify().evalf() 

      
        f = open(result_directory_1 + "INPUT_STRESS_FILE.dat","a+")
        f.write("%6.5f %6.5f %6.5f %6.5f %6.5f \n" % \
                   (x1, y1, sxx,syy,sxy))
        f.close()
        
        f = open(result_directory_1 + "INPUT_STRESS_FILE_unloading.dat","a+")
        f.write("%6.5f %6.5f %6.5f %6.5f %6.5f \n" % \
                   (x1, y1, -sxx,-syy,-sxy))
        f.close()
      #  print(x1,y1,P,sxx,syy,sxy)

/tmp/ipykernel_532245/4000476153.py:41: RuntimeWarning: invalid value encountered in double_scalars
  theta1 = np.arccos(sum1 / dot1)
/tmp/ipykernel_532245/4000476153.py:42: RuntimeWarning: invalid value encountered in double_scalars
  theta0 = np.arccos(sum2 / dot2)
/tmp/ipykernel_532245/4000476153.py:43: RuntimeWarning: invalid value encountered in double_scalars
  theta2 = np.arccos(sum3 / dot3)


In [28]:
### integrant function - integration not required

sxx= P[i]/np.pi*\
        abs((theta0 - 0.5*smp.sin(2*theta0))*(1+(x1/x4))\
            +height[i]/x4*(smp.sin(theta0)*smp.sin(theta0)+2*math.log(smp.cos(theta0)))-\
            (theta1 - 0.5*smp.sin(2*theta1))*(1+(x1/x4))\
            +height[i]/x4*(smp.sin(theta1)*smp.sin(theta1)+2*math.log(smp.cos(theta1))))\
        \
            +P[i]/np.pi*\
        abs((theta2 - 0.5*smp.sin(2*theta2))*(1-(x1/x4))\
            -height[i]/x4*(smp.sin(theta2)*smp.sin(theta2)+2*math.log(smp.cos(theta2)))-\
            (theta0 - 0.5*smp.sin(2*theta0))*(1-(x1/x4))\
            -height[i]/x4*(smp.sin(theta0)*smp.sin(theta0)+2*math.log(smp.cos(theta0)))).simplify().evalf()
        
        syy= P[i]/np.pi*\
        abs((theta0 + 0.5*smp.sin(2*theta0))*(1+(x1/x6))\
            -height[i]/x6*(smp.sin(theta0)*smp.sin(theta0))-\
            (theta1 + 0.5*smp.sin(2*theta1))*(1+(x1/x6))\
            -height[i]/x6*(smp.sin(theta1)*smp.sin(theta1)))\
        \
            +P[i]/np.pi*\
        abs((theta2 + 0.5*smp.sin(2*theta2))*(1-(x1/x4))\
            -height[i]/x4*(smp.sin(theta2)*smp.sin(theta2))-\
            (theta0 + 0.5*smp.sin(2*theta0))*(1-(x1/x4))\
            -height[i]/x4*(smp.sin(theta0)*smp.sin(theta0))).simplify().evalf()
        
        sxy = P[i]/np.pi*\
        abs((smp.sin(theta0)*smp.sin(theta0)*(1+ (x1/x6)))-height[i]/x6*(theta0 -0.5*smp.sin(2*theta0))-\
            (smp.sin(theta1)*smp.sin(theta1)*(1+ (x1/x6)))-height[i]/x6*(theta1 -0.5*smp.sin(2*theta1)))
        \
            +P[i]/np.pi*\
        abs((smp.sin(theta2)*smp.sin(theta2)*(1-(x1/x4)))+height[i]/x4*(theta2 -0.5*smp.sin(2*theta2))-\
            (smp.sin(theta0)*smp.sin(theta0)*(1-(x1/x4)))+height[i]/x4*(theta0 -0.5*smp.sin(2*theta0))).simplify().evalf()
            

IndentationError: unexpected indent (2970959824.py, line 15)